In [1]:
import sys
sys.path.insert(0, '/notebooks/chatweb3')

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser

from chat.base import *
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *

Added to class registry: chat.simple.SimpleChat
Added to class registry: chat.rephrase.RephraseChat
Added to class registry: chat.rephrase_cited.RephraseCitedChat
Added to class registry: chat.widget_search.WidgetSearchChat
Added to class registry: tools.index_lookup.IndexLookupTool
Added to class registry: tools.index_answer.IndexAnswerTool
Added to class registry: tools.index_widget.IndexWidgetTool
Added to class registry: index.weaviate.WeaviateIndex
Added to class registry: tools.index_app_info.IndexAppInfoTool
Added to class registry: chat.basic_agent.BasicAgentChat


In [3]:
set_api_key()

In [4]:
CLEAN_TEXT = '''
RAW TEXT: {text}
---
Remove the irrelevant and uninformative texts from the above paragraph.
CLEAN TEXT:'''

IGNORE_IRRELEVANT_CONTEXT_TEMPLATE = '''
I am interacting with a chatbot and I asked it a question. In order to answer it, the chatbot retrieved some context from the web.  
Now, you are given the question and the scraped content with the corresponding url. You have to respond (explicitly in Yes/No) if the content has an answer to the given question.

---Example---

URL: https://blog.thundercore.com/thundercore-blockchain-partners-with-huobi-mycontainer-in-node-expansion-7b39d4534113
SOURCE: ThunderCore Blockchain Partners with Huobi, MyCointainer in Node Expansion

The Top 5 blockchain ThunderCore is making an ecosystem-development push, partnering with a host of new validators as the chain rolls out its new cryptocurrency staking model. At the same time, ThunderCore is leveraging these promising developments to expand its presence in decentralized finance (DeFi), making overtures to leading protocols.

The newest ThunderCore validators include the famous virtual asset exchange Huobi and one of the earliest staking platforms in the space, MyCointainer. Users of both will now be able to stake ThunderCore’s native asset TT to earn rewards.

Unlike proof-of-work (PoW) blockchains like Bitcoin, ThunderCore is an environmentally friendly proof-of-stake (PoS) chain, which means that its consensus mechanism for validating transactions is based on incentivized crypto staking.

ThunderCore’s new PoS staking model, also known as ThunderCore 2.0, will dramatically increase both user staking and validator nodes. Such decentralization is crucial to secure PoS chains like ThunderCore, which must withstand coordinated attacks by malicious actors seeking to control two-thirds of the network, while adding more trustworthy validators also improves scalability and accelerates user growth.
QUESTION: Which are the top 5 blockchain startups?
RESPONSE: No

---Example---

URL: https://builder.decentraland.org/
SOURCE: Let's build the Metaverse together!
Collections
Create, review and publish your collections of wearables and emotes.

Scenes
Build your space in your LAND. You can use free 3D models or upload your own.

Land
Publish Scenes, create Estates and manage permissions of your LAND.

Names
Claim your unique name and assign it to your avatar or your parcels.
QUESTION: who built you?
RESPONSE: No

---Example---

URL: https://phemex.com/academy/what-is-gnosis-safe#:~:text=Gnosis%20Safe%20is%20a%20non,key%20wallet%2C%20such%20as%20MetaMask
SOURCE: Gnosis Safe is a non-custodial, multisig crypto wallet that evolved from its predecessor, the Gnosis Mutisig wallet.

Conventionally, most crypto users store their crypto in a single key wallet, such as MetaMask. To approve any transactions like sending crypto out of the wallet, a private key, which is generated from that wallet’s unique 12-word seed phrase, is needed. If this private key is compromised or leaked, the crypto can be stolen.

What if your funds are secured not with one but three private keys? This way, even if you lose one key, you can still access your funds with the other two. 

This is what Gnosis Safe’s multisig wallet enables.

Multisig wallets require confirmation from–as the term implies–multiple signatures–that is, several private keys, before a  transaction can be executed.

And if those funds belong to a crypto project or a DAO, the multisig wallet system prevents any single employee from stealing or losing the money, because multiple signatures are needed to approve the transaction.
QUESTION: What is a Gnosis Safe wallet?
RESPONSE: Yes

---Example---

URL: {url}
SOURCE: {source}
QUESTION: {question}
RESPONSE:'''

In [5]:
output_parser = ChatOutputParser()
llm = OpenAI(
    temperature=0.0, max_tokens=-1,
)

text_cleaning_prompt = PromptTemplate(
    input_variables=["text"],
    template=CLEAN_TEXT,
    output_parser=output_parser,
)
text_cleaning_chain = LLMChain(llm=llm, prompt=text_cleaning_prompt, verbose=False)

context_filtering_prompt = PromptTemplate(
    input_variables=["url", "source", "question"],
    template=IGNORE_IRRELEVANT_CONTEXT_TEMPLATE,
    output_parser=output_parser,
)
context_filtering_chain = LLMChain(llm=llm, prompt=context_filtering_prompt, verbose=False)

In [6]:
doc_index = WeaviateIndex('IndexV1', 'content', ['url']) 
widget_index = WeaviateIndex('WidgetV1', 'content')
similarity_config = {
    "properties": [
        {
            "name": "name",
            "weight": 1.0
        }
    ],
    "weights": {
        "name": 1.0
    },
    "similarity": {
        "distance": "Euclidean"
    }
}

In [7]:
question = 'What are some of the advantages of using a decentralized exchange?'
docs = doc_index.similarity_search(question, config=similarity_config)

print("--------question--------")
print(question)
for doc in docs:
    page_content = text_cleaning_chain.apply_and_parse([{"text":doc.page_content}])[0]
    inp = {
        "url": doc.metadata['url'],
        "source": page_content,
        "question": question,
        "stop": "\n",
    }

    result = context_filtering_chain.apply_and_parse([inp])[0]
    if result=="Yes": 
        print("--------selected--------")
    else:
        print("--------discarded--------")
    print(doc.metadata['url'])
    print(page_content)
    print()

--------question--------
What are some of the advantages of using a decentralized exchange?
--------discarded--------
https://docs.dodoex.io/english/dodo-academy/economics-102
The DeFi Summer in mid-2020 kicked off the decentralized marketplaces boom. Trading volumes have soared to levels comparable to centralized exchanges as more technical solutions to liquidity problems have been introduced. In fact, in a letter filed with the SEC in February along with its stock offering statement, Coinbase cited the rise of decentralized exchanges as a key risk to its business model. Innovations in liquidity have helped drive the boom in DEX trading volume. Liquidity mining offers a new asset type for investors willing to build liquidity pools - governance tokens - and incentivizes holders to influence projects. This approach proliferated in 2020, and many platforms are taking a similar approach, allowing liquidity providers to have a greater impact on prices.

--------selected--------
https://vel